This notebook is used to define a noise model for benchmarking purposes.

In [1]:
import numpy as np
import qutip as qt

In [2]:
import qsharp
from qsharp.experimental import enable_noisy_simulation, get_noise_model, set_noise_model
enable_noisy_simulation()

Preparing Q# environment...


In [3]:
I, X, Y, Z = qt.qeye(2), qt.sigmax(), qt.sigmay(), qt.sigmaz()

In [4]:
sI, sX, sY, sZ = map(qt.to_super, [I, X, Y, Z])

In [5]:
def total_dephasing_channel():
    return (1 / 2) * sI + (1 / 2) * sZ

In [6]:
def dephasing_channel(p):
    return (1 - p) * sI + p * total_dephasing_channel()

In [7]:
dephasing_channel(0.1)

Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = (4, 4), type = super, isherm = True
Qobj data =
[[1.  0.  0.  0. ]
 [0.  0.9 0.  0. ]
 [0.  0.  0.9 0. ]
 [0.  0.  0.  1. ]]

In [8]:
def total_depolarizing_channel():
    return (sI + sX + sY + sZ) / 4

In [9]:
def depolarizing_channel(p):
    return (1 - p) * sI + p * total_depolarizing_channel()

In [10]:
def finite_visibility_meas(visibility):
    return [
        qt.to_super(visibility * (I + sign * Z) / 2 + (1 - visibility) * I / 2)
        for sign in (+1, -1)
    ]

In [11]:
noise_model = get_noise_model()

In [12]:
noise_model['x'] = depolarizing_channel(0.1) * sX
noise_model['z'] = dephasing_channel(0.1) * sZ
noise_model['z_meas']['effects'] = finite_visibility_meas(0.95)

In [13]:
set_noise_model(noise_model)
qsharp.client._execute('%experimental.noise_model --save benchmark-noise-model.json')